In [100]:
import random
from OCC.Display.WebGl.x3dom_renderer import *
from IPython.display import display, HTML
%matplotlib inline
caller_id = 0
def DisplayShape(shape,
                     vertex_shader=None,
                     fragment_shader=None,
                     export_edges=False,
                     color=(random.random(), random.random(), random.random()),
                     specular_color=(1, 1, 1),
                     shininess=0.9,
                     transparency=0.,
                     line_color=(0, 0., 0.),
                     line_width=2.,
                     mesh_quality=1.):
        def Show(src="<shape><appearance><material diffuseColor='0.603 0.894 0.909'></material></appearance> <box></box></shape>   ", height=400,  width=400):
            width=str(width)
            height=str(height)
            result = ""
            if(caller_id <2):
                result += " <script type='text/javascript' src='http://www.x3dom.org/download/x3dom.js'> </script>  <link rel='stylesheet' type='text/css' href='http://www.x3dom.org/download/x3dom.css'></link>"
           
            result +="<div style='height: "+height+"px;width: 100%;' id='x3dholder_"+str(caller_id)+"'  width='100%' height='"+height+"px'><x3d style='height: "+height+"px;width: 100%;' id='x3d"+str(caller_id)+"' width='100%' height='"+height+"px'><scene>"+src+"   </scene></x3d> </div>"
            return result
        x3d_exporter = X3DExporter(shape, vertex_shader, fragment_shader,
                                   export_edges, color,
                                   specular_color, shininess, transparency,
                                   line_color, line_width, mesh_quality)
        x3d_exporter.compute()
        tmp = x3d_exporter.to_x3dfile_string()
        temp_file_name = "tmp_"+str(++caller_id)+".x3d"
        if os.path.exists(temp_file_name): os.remove(temp_file_name)
        text_file = open(temp_file_name, "w")
        text_file.write(tmp)
        text_file.close()
        return HTML(Show("<inline url='./"+temp_file_name+"'> </inline> "))
    

import numpy
print(numpy.__path__)

['/opt/conda/lib/python3.5/site-packages/numpy']


In [135]:
from OCC.Display.WebGl import x3dom_renderer
from OCC.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.gp import gp_Vec
from OCC.Display.WebGl.x3dom_renderer import *
from utils import *
from dolfin import *
import os
from OCC.StlAPI import StlAPI_Writer
from OCC.BRepMesh import BRepMesh_IncrementalMesh


my_sphere = BRepPrimAPI_MakeSphere(1).Shape()
geometry = DisplayShape(my_sphere)
display(geometry)

# then mesh it. This mesh is used by the STL exporter
# Note : if the mesh is not performed, the STL exporter
# won't do nothing
mesh = BRepMesh_IncrementalMesh(my_sphere, 0.06)
mesh.Perform()

stl_file = "./cube_low_resolution.stl"
stl_exporter = StlAPI_Writer()
stl_exporter.SetASCIIMode(True)  # change to False if you need binary export
stl_exporter.Write(my_sphere, stl_file)


# then we change the mesh resolution
#mesh.SetDeflection(0.05)


from OCC.Display.SimpleGui import init_display
from OCC.TopoDS import TopoDS_Shape
from OCC.StlAPI import StlAPI_Reader

stl_reader = StlAPI_Reader()
fan_shp = TopoDS_Shape()
stl_reader.Read(fan_shp, stl_file)

exproted = DisplayShape(fan_shp)
display(exproted)

In [136]:
import subprocess
geoCounter = 0
def getGeo(stl_filename):
    geoFile = "Merge \""+stl_filename+"\";\nSurface Loop(1) = {1};\nVolume(1) = {1};\nPhysical Volume(1) = {1};\n"
    temp_file_name = "tmp_"+str(++geoCounter)+".geo"
    if os.path.exists(temp_file_name): os.remove(temp_file_name)
    text_file = open(temp_file_name, "w")
    text_file.write(geoFile)
    text_file.close()
    return temp_file_name;
out = ""
try:
    out = subprocess.check_output(
            ["gmsh", "gmsh -3 -algo pack tmp_0.geo -o SFM.msh"],
            stderr=subprocess.STDOUT
            ).strip().decode('utf8')
except subprocess.CalledProcessError as e:
    out = e.output
    
getGeo(stl_file)
print(out)

b"Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/DN66G33K7VLT36V3EOPQLEWVKT:/var/lib/docker/overlay2/l/NASXZFW5UZSQSPRPWTFPSLGZSB:/var/lib/docker/overlay2/l/3ECOUWQRCKNRCEZG5GXQMRNLTF:/var/lib/docker/overlay2/l/MT2KF5SWYZ3KLOSGTZXDDDTXL6:/var/lib/docker/overlay2/l/EWEHL722SMA77OPCL7Z7PQNDA6:/var/lib/docker/overlay2/l/2TLN6ME67TLH32WHOCBDOTGBJ5:/var/lib/docker/overlay2/l/BKVVY5T4QQM5UXVSSPTXS5GRWW:/var/lib/docker/overlay2/l/TKQIIDSHVJFLRHEPHYY3CAAHAD:/var/lib/docker/overlay2/l/SO2KFZ3ZNW5BZ'\nUnexpected end of /proc/mounts line `6TNOMT7SWI4QT:/var/lib/docker/overlay2/l/KC24YTY2DUOXVESTMWBUNKEELF:/var/lib/docker/overlay2/l/5LZJR6HPC6SOVIDRDUD6FYPSWS:/var/lib/docker/overlay2/l/QJC6JUMCQMEGBE6GOTCCFF3GYJ:/var/lib/docker/overlay2/l/GGYWY3YMFNXLK4F25ZO3SFQZ35:/var/lib/docker/overlay2/l/PAPT7ON6DZMSQHSWBGYLYGWK6G:/var/lib/docker/overlay2/l/2W4UA2QEVRB2OJCHP5VPPOECF7:/var/lib/docker/overlay2/l/JMDH77HDS2KU5VT27MMBAVQHB2,upperdir=/var/lib/

In [137]:
!gmsh -3 algo meshadapt tmp_0.geo -o SFM.msh
!dolfin-convert SFM.msh sfm.xml
!ls -la

Unexpected end of /proc/mounts line `overlay / overlay rw,relatime,lowerdir=/var/lib/docker/overlay2/l/DN66G33K7VLT36V3EOPQLEWVKT:/var/lib/docker/overlay2/l/NASXZFW5UZSQSPRPWTFPSLGZSB:/var/lib/docker/overlay2/l/3ECOUWQRCKNRCEZG5GXQMRNLTF:/var/lib/docker/overlay2/l/MT2KF5SWYZ3KLOSGTZXDDDTXL6:/var/lib/docker/overlay2/l/EWEHL722SMA77OPCL7Z7PQNDA6:/var/lib/docker/overlay2/l/2TLN6ME67TLH32WHOCBDOTGBJ5:/var/lib/docker/overlay2/l/BKVVY5T4QQM5UXVSSPTXS5GRWW:/var/lib/docker/overlay2/l/TKQIIDSHVJFLRHEPHYY3CAAHAD:/var/lib/docker/overlay2/l/SO2KFZ3ZNW5BZ'
Unexpected end of /proc/mounts line `6TNOMT7SWI4QT:/var/lib/docker/overlay2/l/KC24YTY2DUOXVESTMWBUNKEELF:/var/lib/docker/overlay2/l/5LZJR6HPC6SOVIDRDUD6FYPSWS:/var/lib/docker/overlay2/l/QJC6JUMCQMEGBE6GOTCCFF3GYJ:/var/lib/docker/overlay2/l/GGYWY3YMFNXLK4F25ZO3SFQZ35:/var/lib/docker/overlay2/l/PAPT7ON6DZMSQHSWBGYLYGWK6G:/var/lib/docker/overlay2/l/2W4UA2QEVRB2OJCHP5VPPOECF7:/var/lib/docker/overlay2/l/JMDH77HDS2KU5VT27MMBAVQHB2,upperdir=/var/lib/doc

In [138]:

# %load /home/fenics/demo/documented/poisson/python/demo_poisson.py
"""This demo program solves Poisson's equation

    - div grad u(x, y) = f(x, y)

on the unit square with source f given by

    f(x, y) = 10*exp(-((x - 0.5)^2 + (y - 0.5)^2) / 0.02)

and boundary conditions given by

    u(x, y) = 0        for x = 0 or x = 1
du/dn(x, y) = sin(5*x) for y = 0 or y = 1
"""

# Copyright (C) 2007-2011 Anders Logg
#
# This file is part of DOLFIN.
#
# DOLFIN is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# DOLFIN is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
# GNU Lesser General Public License for more details.
#
# You should have received a copy of the GNU Lesser General Public License
# along with DOLFIN. If not, see <http://www.gnu.org/licenses/>.
#
# First added:  2007-08-16
# Last changed: 2012-11-12

# Begin demo

from dolfin import *
from IPython.display import HTML

# Create mesh and define function space
mesh = Mesh("./sfm.xml")
#mesh = UnitCubeMesh(32, 32, 32)
V = FunctionSpace(mesh, "Lagrange", 1)

# Define Dirichlet boundary (x = 0 or x = 1)
def boundary(x):
    return x[0] < DOLFIN_EPS or x[0] > 1.0 - DOLFIN_EPS

# Define boundary condition
u0 = Constant(0.0)
bc = DirichletBC(V, u0, boundary)

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
f = Expression("10*exp(-(pow(x[0] - 0.5, 2) + pow(x[1] - 0.5, 2)) / 0.02)", degree=2)
g = Expression("sin(5*x[0])", degree=2)
a = inner(grad(u), grad(v))*dx
L = f*v*dx + g*v*ds

# Compute solution
u = Function(V)
solve(a == L, u, bc)

# Save solution in VTK format
file = File("poisson.pvd")
file << u

# Plot solution
HTML(X3DOM().html(u))